## 四个主流估计器A股深交所的测试

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sys
sys.path.append('C:/Users/Administrator/Desktop/Repositories/Low-Frequency-Spread-Estimator')
sys.path.append('C:/Users/Handsome Bad Guy/Desktop/Repositories/Low-Frequency-Spread-Estimator')

from SpreadEstimator.SpreadEstimator import SpreadEstimator
from mytools.AutoTester import AutoTester

from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim

from sklearn.linear_model import LinearRegression

In [2]:
se = SpreadEstimator()
univ = ~np.isnan(se.data.data_dic['close'])  # 合法的univ

In [3]:
def get_xy(x,y,univ):
    xx = []
    yy = []
    for i in range(len(x)):
        se = (~np.isnan(x[i])) & univ[i]
        if np.sum(se) == 0:
            continue
        xx.append(x[i:i+1, se].T)
        yy.append(y[i,se])
    return np.vstack(xx), np.hstack(yy)

#### HL, 2012

In [4]:
print(np.sqrt(2)-1)
print(3-np.sqrt(2)*2)

0.41421356237309515
0.1715728752538097


In [15]:
lr = LinearRegression()

beta = 'prod{tsmean{powv{minus{logv{high},logv{low}},2},2},2}'
gamma = 'powv{minus{logv{tsmax{high,2}},logv{tsmin{low,2}}},2}'
alpha = 'div{prod{' + 'powv{' + beta + ',0.5},0.4142},0.1716}'
fml = 'minus{' + alpha + ',' + 'powv{' + 'div{' + gamma + ',0.1716},0.5}}'

fml = 'div{' + 'minus{expv{' + fml + '},1},' + 'add{expv{' + fml + '},1}}'
fml = 'condition{' + 'ge{' + fml +',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',10}'

print('cs_sp')
stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='spread')
x, y = get_xy(signal, se.data.spread, univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('cs_rel_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='cs', spread_type='relative_spread')
x, y = get_xy(signal, se.data.spread_dic['relative_spread'], univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('ts_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='ts', spread_type='spread')
x, y = get_xy(signal, se.data.spread, univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('ts_rel_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='ts', spread_type='relative_spread')
x, y = get_xy(signal, se.data.spread_dic['relative_spread'], univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))

cs_sp
mean corr: 0.1487, positive_corr_ratio: 1.0000, corr_IR: 2.6631
0.02079677296217979

cs_rel_sp
mean corr: -0.1001, positive_corr_ratio: 0.0558, corr_IR: -1.5049
0.0004695213406161181

ts_sp
mean corr: 0.2227, positive_corr_ratio: 0.8596, corr_IR: 1.1170
0.02079677296217979

ts_rel_sp
mean corr: -0.1069, positive_corr_ratio: 0.3361, corr_IR: -0.4168
0.0004695213406161181


#### Roll, 1999

In [16]:
fml = 'neg{tscov{tsdelta{logv{close},1},tsdelay{tsdelta{logv{close},1},1},10}}'

print('cs_sp')
stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread, univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('cs_rel_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='cs', spread_type='relative_spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread_dic['relative_spread'], univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('ts_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='ts', spread_type='spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread, univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('ts_rel_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='ts', spread_type='relative_spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread_dic['relative_spread'], univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))

cs_sp
mean corr: 0.0393, positive_corr_ratio: 0.7241, corr_IR: 0.6569
0.02091495205976779

cs_rel_sp
mean corr: -0.0561, positive_corr_ratio: 0.1940, corr_IR: -0.7631
0.00046702039740793486

ts_sp
mean corr: 0.0387, positive_corr_ratio: 0.5798, corr_IR: 0.2228
0.02091495205976779

ts_rel_sp
mean corr: -0.0172, positive_corr_ratio: 0.4601, corr_IR: -0.0868
0.00046702039740793486


In [17]:
AT = AutoTester()
signal[signal<0] = 0
signal = np.sqrt(signal)
signal[signal <= 0.0] = 0.00

In [18]:
print('cs_sp')
stats = AT.test(signal, se.data.spread, corr_type='linear',method='cs')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
print()

print('cs_rel_sp')
stats = AT.test(signal, se.data.spread_dic['relative_spread'], corr_type='linear',method='cs')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
print()

print('ts_sp')
stats = AT.test(signal, se.data.spread, corr_type='linear',method='ts')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
print()

print('ts_rel_sp')
stats = AT.test(signal, se.data.spread_dic['relative_spread'], corr_type='linear',method='ts')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))

cs_sp
mean corr: 0.0369, positive_corr_ratio: 0.6810, corr_IR: 0.5905

cs_rel_sp
mean corr: -0.0645, positive_corr_ratio: 0.1940, corr_IR: -0.7937

ts_sp
mean corr: 0.0487, positive_corr_ratio: 0.6279, corr_IR: 0.3161

ts_rel_sp
mean corr: -0.0138, positive_corr_ratio: 0.4774, corr_IR: -0.0743


#### CHL

In [22]:
a = 'minus{logv{tsdelay{close,1}},div{add{logv{tsdelay{high,1}},logv{tsdelay{low,1}}},2}}'
b = 'minus{logv{tsdelay{close,1}},div{add{logv{high},logv{low}},2}}'
fml = 'prod{' + a + ',' + b +'}'
fml = 'condition{' + 'ge{' + fml +',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',10}'

print('cs_sp')
stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread, univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('cs_rel_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='cs', spread_type='relative_spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread_dic['relative_spread'], univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('ts_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='ts', spread_type='spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread, univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))
print()

print('ts_rel_sp')
stats, signal = se.test_factor(fml, corr_type='linear',method='ts', spread_type='relative_spread')
signal[signal<0] = 0
signal = np.sqrt(signal)

x, y = get_xy(signal, se.data.spread_dic['relative_spread'], univ)
lr.fit(x, y)
print(np.sqrt(np.mean((y-lr.predict(x))**2)))

cs_sp
mean corr: 0.0795, positive_corr_ratio: 0.9571, corr_IR: 1.6115
0.020884857810250776

cs_rel_sp
mean corr: -0.1688, positive_corr_ratio: 0.0172, corr_IR: -2.2806
0.0004638988812102302

ts_sp
mean corr: 0.1840, positive_corr_ratio: 0.8183, corr_IR: 0.9083
0.020884857810250776

ts_rel_sp
mean corr: -0.1072, positive_corr_ratio: 0.3293, corr_IR: -0.4487
0.0004638988812102302


#### LOT_Mix

我们使用torch来进行优化求解

In [116]:
class LOTNet(nn.Module):
    def __init__(self, sample_num: int = 1):
        """
        :param sample_num: 样本数量
        """
        super(LOTNet, self).__init__()
        self.sample_num = sample_num
        self.sigma = nn.Parameter(0.01 * torch.ones(sample_num))  
        self.beta = nn.Parameter(torch.ones(1, sample_num))
        self.alpha_1 = nn.Parameter(0.01 * torch.ones(1, sample_num))
        self.alpha_2 = nn.Parameter(-0.01 * torch.ones(1, sample_num))
        
    def forward(self, R, R_m):  # 传入的R是seq_length * sample_num，R_m是seq_length * 1
        R_1 = R + self.alpha_1.repeat(len(R), 1) - torch.matmul(R_m, self.beta)
        R_2 = R + self.alpha_2.repeat(len(R), 1) - torch.matmul(R_m, self.beta)
        R_t = torch.where(R>0, R_1, R_2)
        R_t = torch.sum(R_t**2, dim=0) / (2 * self.sigma ** 2)
        # print(R_t.shape)
        return -len(R)/2 * torch.log(1e-8+self.sigma**2) - R_t
    
class MyLoss(nn.Module):
    def __init__(self):
        super(MyLoss, self).__init__()
        pass
    
    def forward(self, x):
        return -torch.mean(x)

In [181]:
# 获得R_m和每个股票的日收益率
fml = 'tspct{close,1}'
stats, ret = se.test_factor(fml, corr_type='linear',method='cs', spread_type='spread')
ret = ret.astype(np.float32)
ret[np.isnan(ret)] = 0
R_m = np.zeros(len(ret), dtype=np.float32)
R_m[1:] = np.nanmean(ret[1:], axis=1)

mean corr: 0.0172, positive_corr_ratio: 0.6364, corr_IR: 0.2172


In [286]:
device = 'cuda'
model = LOTNet(2081).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0)
loss_func = MyLoss()

r_m = torch.Tensor(R_m).unsqueeze(-1).to(device)
r = torch.Tensor(ret).to(device)

# R_m = torch.Tensor(R_m[1:51]).unsqueeze(-1).to(device)
# R = torch.Tensor(ret[1:51]).to(device)

signal = np.zeros((240, 2081))
t = time()
for j in range(51, 240):
    for i in range(200):
        model.train()
        optimizer.zero_grad()
        loss = loss_func(model(r[j-20:j+1], r_m[j-20:j+1]))
        loss.backward()
        optimizer.step()
#         if (i + 1) % 100 == 0:
#             print('epoch {}'.format(i + 1))
#             print('loss: {:.4f}, time used: {:.4f}s'.format(float(loss), time()-t))
    model.eval()
    signal[j] = (model.alpha_1 - model.alpha_2).detach().cpu().numpy()
    print('{} done. time used: {:.4f}s'.format(j, time()-t))

51 done. time used: 0.3319s
52 done. time used: 0.6282s
53 done. time used: 0.9141s
54 done. time used: 1.1927s
55 done. time used: 1.4943s
56 done. time used: 1.7759s
57 done. time used: 2.0609s
58 done. time used: 2.3618s
59 done. time used: 2.6594s
60 done. time used: 2.9440s
61 done. time used: 3.2289s
62 done. time used: 3.5184s
63 done. time used: 3.8048s
64 done. time used: 4.0938s
65 done. time used: 4.3832s
66 done. time used: 4.6760s
67 done. time used: 4.9611s
68 done. time used: 5.2457s
69 done. time used: 5.5441s
70 done. time used: 5.8286s
71 done. time used: 6.1160s
72 done. time used: 6.4078s
73 done. time used: 6.7027s
74 done. time used: 6.9825s
75 done. time used: 7.2663s
76 done. time used: 7.5606s
77 done. time used: 7.8439s
78 done. time used: 8.1246s
79 done. time used: 8.4140s
80 done. time used: 8.7001s
81 done. time used: 8.9874s
82 done. time used: 9.2884s
83 done. time used: 9.5817s
84 done. time used: 9.8697s
85 done. time used: 10.1627s
86 done. time used:

In [287]:
print('cs_sp')
stats = AT.test(signal[51:], se.data.spread[51:240], corr_type='linear',method='cs')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal[51:], se.data.spread[51:240], univ[51:240])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

print('cs_rel_sp')
stats = AT.test(signal[51:], se.data.spread_dic['relative_spread'][51:240], corr_type='linear',method='cs')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal[51:], se.data.spread_dic['relative_spread'][51:240], univ[51:240])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

print('ts_sp')
stats = AT.test(signal[51:], se.data.spread[51:240], corr_type='linear',method='ts')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal[51:], se.data.spread[51:240], univ[51:240])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

print('ts_rel_sp')
stats = AT.test(signal[51:], se.data.spread_dic['relative_spread'][51:240], corr_type='linear',method='ts')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal[51:], se.data.spread_dic['relative_spread'][51:240], univ[51:240])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

cs_sp
mean corr: -0.0374, positive_corr_ratio: 0.0265, corr_IR: -1.8764
0.0004980314737797513

cs_rel_sp
mean corr: 0.0610, positive_corr_ratio: 0.9947, corr_IR: 1.7394
2.9178905838578435e-07

ts_sp
mean corr: -0.0746, positive_corr_ratio: 0.3426, corr_IR: -0.4024
0.0004980314737797513

ts_rel_sp
mean corr: 0.0834, positive_corr_ratio: 0.6386, corr_IR: 0.3713
2.9178905838578435e-07



#### Gibbs

In [216]:
import estimator
importlib.reload(estimator.Gibbs2004)
from estimator.Gibbs2004 import gibbs

In [224]:
close = se.data.data_dic['close'].copy()
for i in range(len(close)):
    for j in range(close.shape[1]):
        if np.isnan(close[i, j]):
            if ~np.isnan(close[i-1,j]):
                close[i, j] = close[i-1, j]
close[np.isnan(close)] = 0

In [226]:
t  = time()
c_s, q_s, sigma_u_s = gibbs(close[-20:,:10], 
                            sigma_c=1e6, ig_alpha=2, 
                            ig_beta=1e-4, sample_num=100)
print('done. time used: {:4f}s'.format(time()-t))

done. time used: 0.488182s


In [243]:
# close合法的top
top = np.sum(np.isnan(se.data.data_dic['close']),axis=0)<=15

In [298]:
signal_gibbs = np.zeros((210,np.sum(top)))
for i in range(10):
    c = np.load('../estimator/cache/{}.npy'.format(i))
    signal_gibbs[21*i:21*i+21] = c

In [299]:
print('cs_sp')
stats = AT.test(signal_gibbs, se.data.spread[20:230,top], corr_type='linear',method='cs')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal_gibbs, se.data.spread[20:230,top], univ[20:230,top])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

print('cs_rel_sp')
stats = AT.test(signal_gibbs, se.data.spread_dic['relative_spread'][20:230,top], corr_type='linear',method='cs')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal_gibbs, se.data.spread_dic['relative_spread'][20:230,top], univ[20:230,top])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

print('ts_sp')
stats = AT.test(signal_gibbs, se.data.spread[20:230,top], corr_type='linear',method='ts')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal_gibbs, se.data.spread[20:230,top], univ[20:230,top])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))
print()

print('ts_rel_sp')
stats = AT.test(signal_gibbs, se.data.spread_dic['relative_spread'][20:230,top], corr_type='linear',method='ts')
print('mean corr: {:.4f}, positive_corr_ratio: {:.4f}, corr_IR: {:.4f}'.
              format(stats.mean_corr, stats.positive_corr_ratio, stats.corr_IR))
x, y = get_xy(signal_gibbs, se.data.spread_dic['relative_spread'][20:230,top], univ[20:230,top])
lr.fit(x, y)
print(np.mean((y-lr.predict(x))**2))

cs_sp
mean corr: 0.0019, positive_corr_ratio: 0.5333, corr_IR: 0.0724
0.0005362826888880429

cs_rel_sp
mean corr: 0.0020, positive_corr_ratio: 0.5333, corr_IR: 0.0820
1.7771821828633719e-07

ts_sp
mean corr: 0.0009, positive_corr_ratio: 0.4946, corr_IR: 0.0125
0.0005362826888880429

ts_rel_sp
mean corr: -0.0022, positive_corr_ratio: 0.4832, corr_IR: -0.0310
1.7771821828633719e-07
